## 1 - Attention as Matrix Multiplication
<hr>

Previously, we built a mask vector to help us focus our attention on specific parts of a sequence. How do transformers find the relevant mask vectors for attention? It would be natural to use some sort of a lookup table as given below:

<img src="images/mask_matrix_lookup.png" width=600>

In the matrix showing the collection of mask vectors, we've only shown the one we're trying to pull out, for clarity.

This mask lookup is represented by the $Q K^T$ term in the attention equation.

<img src="images/attention_equation_QKT.png" width=300>

The query $Q$ represents the feature of interest and the matrix $K$ represents the collection of masks. Because it's stored with masks in columns, rather than rows, it needs to be transposed (with the $T$ operator) before multiplying.

## 2 - Second Order Sequence Model as Matrix Multiplications
<hr>

Once we have the result of our attention step, a vector that includes the most recent word and a small collection of the words that have preceded it, we need to translate that into features, each of which is a word pair. Attention masking gets us the raw material that we need, but it doesn't build those word pair features. To do that, we can use a single layer fully connected neural network.

To see how a neural network layer can create these pairs, we'll hand craft one. It will be artificially clean and stylized, and its weights will bear no resemblance to the weights in practice, but it will demonstrate how the neural network has the expressivity necessary to build these two word pair features. To keep it small and clean, will focus on just the three attended words from this example, *battery, program, ran.*

<img src="images/feature_creation_layer.png" width=400>

In the layer diagram above, we can see how the weights act to combine the presence and absence of each word into a collection of features. This can also be expressed in matrix form.

<img src="images/feature_creation_matrix.png" width=300>

And it can be calculated by a matrix multiplication with a vector representing the collection of words seen so far.

<img src="images/second_order_feature_battery.png" width=400>

The *battery* and *run* elements are 1 and the *program* element is 0. The *bias* element is always 1, a feature of neural networks. Working through the matrix multiplication gives a 1 for the element representing *battery, ran* and a -1 for the element representing *program, ran.* The results for the other case are similar.

<img src="images/second_order_feature_program.png" width=400>

The final step in calculating these word combo features is to apply a rectified linear unit (ReLU) nonlinearity. The effect of this is to substitute any negative value with a zero. This cleans up both of these results so they represent the presence (with a 1) or absence (with a 0) of each word combination feature.

<font color="gray"><i>With those gymnastics behind us, we finally have a matrix multiplication based method for creating multiword features. A closer look at this method shows that it can build other features too. When the feature creation matrix is learned, rather than hard coded, other structures can be learned. Even in this toy example, there's nothing to stop the creation of a three-word combination like (battery, program, ran).</i></font>

In this form, the multiword feature matrix is ready for one more matrix multiplication, the second order sequence model with skips we developed previously. All together, the sequence of

- feature creation matrix multiplication,
- ReLU nonlinearity, and
- transition matrix multiplication

are the **feedforward** processing steps that get applied after attention is applied. Equation 2 from the paper shows these steps in a concise mathematical formulation.

<img src="images/feedforward_equations.png" width=500>

The Figure 1 architecture diagram of the of the paper shows these lumped together as the Feed Forward block.

<img src="images/architecture_feedforward.png" width=450>



## 3 - Sequence Completion
<hr>

So far we've only talked about next word prediction. There are a couple of pieces we need to add to get our decoder to generate a long sequence. The first is a **prompt,** some example text to give the transformer a running start and context on which to build the rest of the sequence. It gets fed in to decoder, the column on the right in the image above, where it's labeled "Outputs (shifted right)". Choosing a prompt that gives interesting sequences is an art in itself, called prompt engineering.

Once the decoder has a partial sequence to get started with, it takes a forward pass. The end result is a set of predicted probability distributions of words, one probability distribution for each position in the sequence. We choose the word with the highest probability. The new next word then gets added to the sequence.

The one piece we're not quite ready to describe in detail is yet another form of masking, ensuring that when the transformer makes predictions it only looks behind, not ahead. It's applied in the block labeled "Masked Multi-Head Attention". We'll revisit this later.

## 4 - Positional Encoding
<hr>

One critical aspect of self-attention is that it doesn't inherently capture the sequential order of elements in the input sequence, as it computes attention based on content alone. To address this limitation, positional encodings are added to the input embeddings in transformers. These encodings provide the model with information about the positions of words in the sequence, enabling it to distinguish between words with the same content but different positions.

## 5 - De-embeddings
<hr>

Embedding words makes them vastly more efficient to work with, but once the party is over, they need to be converted back to words from the original vocabulary. De-embedding is done the same way embeddings are done, with a projection from one space to another, that is, a matrix multiplication.

The de-embedding matrix is the same shape as the embedding matrix, but with the number of rows and columns flipped. The number of rows is the dimensionality of the space we're converting from. The de-embedding transform is shown as the Linear block in the above figure.

## 6 - Multi Head Attention
<hr>

- $N$: vocabulary size. Typically in the tens of thousands.
- $n$: maximum sequence length. Something like a few hundred in the paper (not specified). 2048 in GPT-3.
- $\text{d_model}$: number of dimensions in the embedding space used throughout the model. 512 in the paper.

The original input matrix is constructed by getting each of the words from the sentence in their one-hot representation, and stacking them such that each of the one-hot vectors is its own row. The resulting input matrix has $n$ rows and $N$ columns, which we can abbreviate as $[n \times N]$.

<img src="images/matrix_multiply_shape.png" width=600>

We can follow the changes in matrix shape through the transformer as a way to tracking what's going on. After the initial embedding, the positional encoding is additive, rather than a multiplication, so it doesn't change the shape of things. Then the embedded word sequence goes into the attention layers, and comes out the other end in the same shape. (We'll come back to the inner workings of these in a second.) Finally, the de-embedding restores the matrix to its original shape, offering a probability for every word in the vocabulary at every position in the sequence.

<img src="images/matrix_shapes.png" width=500>

### 6.1 - Why We Need More Than One Attention Head

Words are represented as dense embedded vectors, rather than one-hot vectors. Attention isn't just 1 or 0, on or off, but can also be anywhere in between. To get the results to fall between 0 and 1, we use the softmax trick again. It has the dual benefit of forcing all the values to lie in our $[0, 1]$ attention range, and it helps to emphasize the highest value, while agressively squashing the smallest.

A complicating consequence of putting a softmax function in attention is that it will tend to focus on a single element. This is a limitation we didn't have before. Sometimes it's useful to keep several of the preceding words in mind when predicting the next, and the softmax just robbed us of that. This is a problem for the model.

The solution is to have several different instances of attention, or heads running at once. This lets the transformer consider several previous words simultaneously when predicting the next. Unfortunately, doing this really increases the computational load. Computing attention was already the bulk of the work, and we just multiplied it by however many heads we want to use. To get around this, we can re-use the trick of projecting everything into a lower-dimensional embedding space. This shrinks the matrices involved which dramatically reduces the computation time. The day is saved.

To see how this plays out, we can continue looking at matrix shapes. Tracing the matrix shape through the branches and weaves of the multihead attention blocks requires three more numbers.

- $d_k$: dimensions in the embedding space used for keys and queries. 64 in the paper.
- $d_v$: dimensions in the embedding space used for values. 64 in the paper.
- $h$: the number of heads. 8 in the paper.

<img src="images/architecture_multihead.png" width=600>

The $[n \times \text{d_model}]$ sequence of embedded words serves as the basis for everything that follows. In each case there is a matrix, $W_v, W_q$, and $W_k$, (all shown unhelpfully as "Linear" blocks in the architecture diagram) that transforms the original sequence of embedded words into the values matrix, $V$, the queries matrix, $Q$, and the keys matrix, $K$. 

$K$ and $Q$ have the same shape, $[n \times d_k]$, but $V$ can be different, $[n \times d_v]$. It confuses things a little that $d_k$ and $d_v$ are the same in the paper, but they don't have to be. An important aspect of this setup is that each attention head has its own $W_v, W_q$, and $W_k$ transforms. That means that each head can zoom in and expand the parts of the embedded space that it wants to focus on, and it can be different than what each of the other heads is focusing on.

The result of each attention head has the same shape as $V$. Now we have the problem of $h$ different result vectors, each attending to different elements of the sequence. To combine these into one, we exploit the powers of linear algebra, and just concatenate all these results into one giant $[n \times h * d_v]$ matrix. Then, to make sure it ends up in the same shape it started, we use one more transform with the shape $[h * d_v \times \text{d_model}]$.

## 7 - Self Attention
<hr>

<img src="images/selfattention.png" width=800>

The queries and keys matrices, $Q$ and $K$, both come in with shape $[n \times d_k]$. Thanks to $K$ being transposed before multiplication, the result of $Q K^T$, gives a matrix of $[n \times d_k] * [d_k \times n ] = [n \times n]$. 

**Dividing every element of this matrix by the square root of $d_k$ has been shown to keep the magnitude of the values from growing wildly, and helps backpropagation to perform well.**

The softmax tends to focus attention on one element of the sequence more than the rest. In this form, the $[n \times n]$ attention matrix roughly maps each element of the sequence to one other element of the sequence, indicating what it should be watching in order to get the most relevant context for predicting the next element. It is a filter that finally gets applied to the values matrix $V$, leaving only a collection of the attended values. This has the effect of ignoring the vast majority of what came before in the sequence, and shines a spotlight on the one prior element that is most useful to be aware of.

$$\text{Attention}(Q, K, V) = \text{softmax} \left( \frac{QK^T}{\sqrt{d_k}} \right) V $$

One tricky part about understanding this set of calculations is keeping in mind that it is calculating attention for every element of our input sequence, for every word in our sentence, not just the most recent word. It's also calculating attention for earlier words. We don't really care about these because their next words have already been predicted and established. It's also calculating attention for future words. These don't have much use yet, because they are too far out and their immediate predecessors haven't yet been chosen. But there are indirect paths through which these calculations can effect the attention for the most recent word, so we include them all. It's just that when we get to the end and calculate word probabilities for every position in the sequence, we throw away most of them and only pay attention to the next word.

The Mask block enforces the constraint that, at least for this sequence completion task, we can't look into the future. It avoids introducing any weird artifacts from imaginary future words. It is crude and effective - manually set the attention paid to all words past the current position to negative infinity. Purple cells show where attention is disallowed. Each row corresponds to an element in the sequence. The first row is allowed to attend to itself (the first element), but to nothing after. The last row is allowed to attend to itself (the final element) and everything that comes before. The Mask is an $[n \times n]$ matrix. It is applied not with a matrix multiplication, but with a more straightforward element-by-element multiplication. This has the effect of manually going in to the attention matrix and setting all of the purple elements from the mask to negative infinity.

<img src="images/mask.png" width=300>



### References
The contents of this notebook are based on the following article:

https://e2eml.school/transformers